In [1]:
! pip install ollama

  Using cached ollama-0.4.7-py3-none-any.whl (13 kB)



[notice] A new release of pip available: 22.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from ollama import chat
from ollama import ChatResponse

# response: ChatResponse = chat(model='llama3.2', messages=[
#   {
#     'role': 'user',
#     'content': 'Why is the sky blue?',
#   },
# ])
# print(response['message']['content'])
# or access fields directly from the response object
# print(response.message.content)

In [ ]:
prompt = """You are a helpful copy writer for an e commerce company that uploads products to the site. 
Given an one line description of a product write a complete content about the product that can be used as an mock data.
The generated content must include three sections:
    Product Description (Paragraph)
    Features: Salient Features of the product (List)
    Dimensions: List down the dimensions of the product if applicable (user provided or made up values)

Just create the content dont add any sentence at the beginning  
"""

stream: ChatResponse = chat(model='llama3.2', messages=[
  {
    'role': 'user',
    'content': prompt,
  },
],
  stream=True,
)

for chunk in stream:
  print(chunk['message']['content'], end='', flush=True)

**Product Name:** SmartWatch Fusion X

The SmartWatch Fusion X is an innovative wearable device designed to transform your daily routine into a seamless experience. With its sleek and modern design, this smartwatch seamlessly blends style with functionality, ensuring that you stay connected, on top of your game, and in sync with the world around you.

**Features:**

• Water-resistant up to 50 meters for an immersive aquatic experience
• Long-lasting battery life of up to 7 days, allowing for uninterrupted wear
• Advanced health monitoring features, including heart rate tracking, sleep quality analysis, and stress level detection
• Wireless charging capabilities for a hassle-free recharging process
• Compatible with both iOS and Android devices for seamless connectivity

**Dimensions:**

- Length: 47mm
- Width: 33.5mm
- Height: 11.4mm

In [3]:
from ollama import chat

stream = chat(
    model='llama3.2',
    messages=[{'role': 'user', 'content': 'Why is the sky blue?'}],
    stream=True,
)

for chunk in stream:
  print(chunk['message']['content'], end='', flush=True)

The sky appears blue because of a phenomenon called Rayleigh scattering, named after the British physicist Lord Rayleigh, who first described it in the late 19th century.

Here's what happens:

1. When sunlight enters Earth's atmosphere, it encounters tiny molecules of gases such as nitrogen (N2) and oxygen (O2).
2. These molecules scatter the light in all directions, but they scatter shorter (blue) wavelengths more than longer (red) wavelengths.
3. This is because the smaller molecules are more effective at scattering the shorter wavelengths due to their size and density.
4. As a result, the blue light is dispersed in all directions, reaching our eyes from all parts of the sky.
5. Our brains interpret this scattered blue light as the color of the sky.

The reason why we don't see the sky as a uniform blue is because of the Earth's atmosphere. The atmosphere scatters light in different ways, depending on its composition and temperature. For example:

* At sunrise and sunset, the sky ca

In [10]:
from ollama import chat

stream = chat(
    model='llama3.2',
    messages=[{'role': 'user', 'content': 'What is 77*77/88? Do the calculation step by step and try to be precise'}],
    stream=True,
)

for chunk in stream:
  print(chunk['message']['content'], end='', flush=True)

To calculate this, I'll follow the order of operations (PEMDAS):

1. Multiply 77 and 77:
77 * 77 = 5929
2. Divide the result by 88:
5929 ÷ 88 ≈ 67.25

So, 77*77/88 ≈ 67.25

## Pydantic

- Introduce llm from there
- Introduce prompt templates
- Introduce strucutured response raw
- Structured response json mode
- Structured response Pydantic

In [ ]:
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAI
from pydantic import BaseModel, Field, model_validator

model = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0.0)


# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

    # You can add custom validation logic easily with Pydantic.
    @model_validator(mode="before")
    @classmethod
    def question_ends_with_question_mark(cls, values: dict) -> dict:
        setup = values.get("setup")
        if setup and setup[-1] != "?":
            raise ValueError("Badly formed question!")
        return values


# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# And a query intended to prompt a language model to populate the data structure.
prompt_and_model = prompt | model
output = prompt_and_model.invoke({"query": "Tell me a joke."})
parser.invoke(output)